In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import seaborn as sns

from natsort import natsorted
import GEOparse as geo
from glob import glob
import os
from os.path import basename, splitext

# GSE124872 (RNA)

In [6]:
gse_id = 'GSE124872'
gse_type = 'RNA'
gse_path = '/tank/projects/kriukov_clocks/' + gse_id + '_' + gse_type

try:
    gse = geo.get_GEO(geo=gse_id, destdir=gse_path)
except:
    gse = geo.get_GEO(geo=gse_id, destdir=gse_path)

09-Apr-2022 16:39:06 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE124nnn/GSE124872/soft/GSE124872_family.soft.gz to /tank/projects/kriukov_clocks/GSE124872_RNA/GSE124872_family.soft.gz
100%|██████████| 5.14k/5.14k [00:00<00:00, 8.87kB/s]
09-Apr-2022 16:39:07 DEBUG downloader - Size validation passed
09-Apr-2022 16:39:07 DEBUG downloader - Moving /tmp/tmpz_7nlgj1 to /tank/projects/kriukov_clocks/GSE124872_RNA/GSE124872_family.soft.gz
09-Apr-2022 16:39:07 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE124nnn/GSE124872/soft/GSE124872_family.soft.gz
09-Apr-2022 16:39:07 INFO GEOparse - Parsing /tank/projects/kriukov_clocks/GSE124872_RNA/GSE124872_family.soft.gz: 
09-Apr-2022 16:39:07 DEBUG GEOparse - DATABASE: GeoMiame
09-Apr-2022 16:39:07 DEBUG GEOparse - SERIES: GSE124872
09-Apr-2022 16:39:07 DEBUG GEOparse - PLATFORM: GPL21103
09-Apr-2022 16:39:07 DEBUG GEOparse - SAMPLE: GSM3557675
09-Apr-2022 16:39:07 DEBUG GEOparse - SAMPLE

In [30]:
links = gse.metadata['supplementary_file']
links 

In [8]:
#save phenotype annotation data
#gse.phenotype_data.to_csv(gse_path + '/{0}_metadata.csv'.format(gse_id)) #wrong metadata

In [11]:
#download files from supplementary table of GSE
for file_link in links:
    print(file_link)
    os.system("wget {0} --directory-prefix={1}".format(file_link, gse_path))

ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE124nnn/GSE124872/suppl/GSE124872_Angelidis_2018_metadata.csv.gz
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE124nnn/GSE124872/suppl/GSE124872_raw_counts_flow_sorted_bulk.txt.gz
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE124nnn/GSE124872/suppl/GSE124872_raw_counts_single_cell.RData.gz
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE124nnn/GSE124872/suppl/GSE124872_raw_counts_single_cell.mtx.gz
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE124nnn/GSE124872/suppl/GSE124872_raw_counts_whole_lung_bulk.txt.gz


In [ ]:
#unpack the if needed
for f in glob(gse_path + '/*'):
    os.system("tar -xvf {0} -C {1}".format(f, gse_path))

In [12]:
#decompress the if needed gz files
for f in glob(gse_path + '/*.gz'):
    os.system("gzip -d {0}".format(f))

In [14]:
#look at the files in the directory
!ls /tank/projects/kriukov_clocks/GSE124872_RNA

GSE124872_Angelidis_2018_metadata.csv
GSE124872_family.soft
GSE124872_metadata.csv
GSE124872_raw_counts_flow_sorted_bulk.txt
GSE124872_raw_counts_single_cell.RData
GSE124872_raw_counts_single_cell.mtx
GSE124872_raw_counts_whole_lung_bulk.txt


In [18]:
from scipy.io import mmread #open .mtx file
M = mmread(gse_path + '/' + 'GSE124872_raw_counts_single_cell.mtx')
M

In [31]:
meta = pd.read_csv(gse_path + '/' +'GSE124872_Angelidis_2018_metadata.csv', index_col=0)

In [37]:
M.sum(0) #compare with nUMI from the metadata

matrix([[10691,  6860,  6127, ...,   436,  1081,  4433]])

In [32]:
meta

,nGene,nUMI,orig.ident,identifier,res.2,identifier.1,name,grouping,batch,cells,cluster,celltype
muc3838:muc3838:TTCCGTGCCCCT,4255,10691,merged,muc3838,2,muc3838,NaN,24m,good,800,cluster2,Ciliated_cells
muc3838:muc3838:TTGCCCAATTAA,3178,6860,merged,muc3838,2,muc3838,NaN,24m,good,800,cluster2,Ciliated_cells
muc3838:muc3838:AAGCCCAGCTAT,1470,6127,merged,muc3838,24,muc3838,NaN,24m,good,800,cluster24,Plasma_cells
muc3838:muc3838:GCACTTTAGAAT,2348,4359,merged,muc3838,2,muc3838,NaN,24m,good,800,cluster2,Ciliated_cells
muc3838:muc3838:TCCTGCTCCCTT,2439,5293,merged,muc3838,2,muc3838,NaN,24m,good,800,cluster2,Ciliated_cells
...,...,...,...,...,...,...,...,...,...,...,...,...
muc4657:muc4657:ATCAAGACAGTG,340,670,merged,muc4657,0,muc4657,NaN,3m,good,1000,cluster0,Type_2_pneumocytes
muc4657:muc4657:GACTGCGCATGG,223,263,merged,muc4657,2,muc4657,NaN,3m,good,1000,cluster2,Ciliated_cells
muc4657:muc4657:ATGACCGAATGT,240,436,merged,muc4657,0,muc4657,NaN,3m,good,1000,cluster0,Type_2_pneumocytes
muc4657:muc4657:GTGTTTGGACCG,668,1081,merged,muc4657,9,muc4657,NaN,3m,good,1000,cluster9,Cd4+_T_cells
